# Power diffraction action

In [ ]:
from SSP import *

In [ ]:
# Import the data, noting the very nonstandard .x_y file format, which requires read engine specification
data = pd.read_csv('neutronscatter.x_y', delim_whitespace=True, engine = 'python', names=['Angle', 'Intensity'])
data['Intensity'] = data['Intensity']/data['Intensity'].max() # Normalise the intensity

# Plot the data

fig, ax = plt.subplots()

ax.plot(data['Angle'], data['Intensity'], label = r'$mv^2/\beta$: 7.47 keV', color = 'C0')
ax.set_xlabel(r'Angle [2$\theta$]')
ax.set_ylabel('Intensity [arb]')
ax.set_title('X-ray scattering of unknown material');
plt.legend()

ax.set_xlim((19,82))

loc = matplotlib.ticker.MultipleLocator(1)
ax.xaxis.set_minor_locator(loc)
ax.tick_params(which='minor')

ax.set_aspect(30) # if adjusting the aspect ratio, make sure to use bbox_inches='tight' when saving!

if False:
    plt.savefig('neutronscatter.pdf', facecolor='white', transparent=False, bbox_inches='tight')

plt.show()

In [ ]:
# Find the peaks
peaks, _ = find_peaks(data['Intensity'], prominence = 0.01)

# Plot the data with peaks

fig, ax = plt.subplots()

ax.plot(data['Angle'], data['Intensity'], label = r'$mv^2/\beta$: 7.47 keV', color = 'C0')
ax.plot(data['Angle'][peaks], data['Intensity'][peaks], "x", color = 'C1', label = 'Peaks')
ax.set_xlabel(r'Angle [2$\theta$]')
ax.set_xlim((19,82))
ax.set_ylabel('Intensity [arb]')
ax.set_title('X-ray scattering of unknown material');
plt.legend()

plt.show()

In [ ]:
analysis = pd.DataFrame() # Make a dataframe for the results

analysis['2theta'] = data['Angle'][peaks] # Add the diffraction angles
analysis.reset_index(drop = True, inplace = True) # Clean the index - just to make things pretty
analysis.index += 1 # Make index the peak number

momenergy = 7477.72 # momenergy - mv^2/\beta of neutrons [eV]

wl = const.h * const.c / (momenergy * e) # Convert energy to wavelength [m]
analysis['d'] = wl / (2 * np.sin(np.deg2rad(analysis['2theta']/2))) # Calculate the plane spacing from Bragg condition [m]
d_a = analysis['d'].iloc[0] # First diffraction peak (d_a)
analysis['ratio'] = (d_a/analysis['d']) ** 2 # Ratio of angles of first peak to subsequent peaks
analysis

In [ ]:
# Look for Miller indices

guess = 8
analysis['N guess'] = (guess * analysis['ratio'])
analysis['N int'] = analysis['N guess'].round(0).astype(int)
analysis['N error'] = 100 * (analysis['N guess'] - analysis['N int'])/analysis['N guess']
analysis